In [1]:
import importlib
import pandas as pd
import geopandas as gpd
import numpy as np
import helper
import calendar
import os, sys
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import style
style.use('ggplot')
PLT_FIG_WIDTH = 4.487
PLT_FIG_HEIGHT = PLT_FIG_WIDTH / 1.618

import st_visualizer
import express as viz_express
import geom_helper as viz_helper
import bokeh.palettes as bokeh_palettes

importlib.reload(helper)
importlib.reload(viz_helper)
import psycopg2
import psycopg2.extras

In [2]:
%%time

df = pd.read_csv("./data/unipi_ais_clean.csv")

df.sort_values('timestamp', inplace=True)

df = helper.getGeoDataFrame_v2(df, crs='epsg:4326')

Wall time: 5min 14s


In [3]:
df.head()

,timestamp,type,mmsi,status,lon,lat,heading,turn,speed,course,timestamp_sec,velocity,bearing,acceleration,geom
0,1519855200000,NaN,636013190,NaN,23.539338,37.885675,44.0,NaN,1.4,15.5,1.519855e+09,1.384017,24.892345,-0.019672,POINT (23.53934 37.88567)
1,1519855201000,NaN,239550200,NaN,23.640850,37.947157,264.0,NaN,0.0,142.1,1.519855e+09,0.265630,77.088586,-0.016206,POINT (23.64085 37.94716)
2,1519855201000,NaN,239722800,NaN,23.632205,37.943968,NaN,NaN,0.0,0.0,1.519855e+09,0.092528,108.961741,0.002525,POINT (23.63220 37.94397)
3,1519855201000,NaN,237991700,NaN,23.546150,37.949950,NaN,NaN,0.0,268.0,1.519855e+09,0.000000,0.000000,0.000000,POINT (23.54615 37.94995)
4,1519855201000,NaN,241024000,NaN,23.650097,37.931657,332.0,NaN,0.0,178.1,1.519855e+09,0.154236,108.908983,0.006314,POINT (23.65010 37.93166)


In [2]:
con = psycopg2.connect(database="gis",
                       user="postgres",
                       password="77889955",
                       host="localhost",
                       port=5432)

script = "select * from world_ports"

ports_wdf = gpd.GeoDataFrame.from_postgis(script, con, geom_col="geom")

In [3]:
ports_wdf.head()

,id,geom,index_no,region_no,port_name,country,latitude,longitude,lat_deg,lat_min,...,elecrepair,provisions,water,fuel_oil,diesel,decksupply,eng_supply,repaircode,drydock,railway
0,981,POINT (23.40000 37.96667),42160.0,42010.0,MEGARA OIL TERMINAL,GR,37.966667,23.400000,37.0,58.0,...,None,None,None,None,None,None,None,None,None,None
1,982,POINT (23.53333 37.98333),42180.0,42010.0,AYIOS NIKOLAOS,GR,37.983333,23.533333,37.0,59.0,...,None,None,None,None,None,None,None,A,S,None
2,983,POINT (23.55000 38.03333),42200.0,42010.0,ELEVSIS,GR,38.033333,23.550000,38.0,2.0,...,Y,Y,Y,Y,Y,Y,Y,A,M,S
3,984,POINT (23.38333 37.96667),42205.0,42010.0,PAKHI OIL TERMINAL,GR,37.966667,23.383333,37.0,58.0,...,None,None,N,Y,Y,None,None,N,None,None
4,985,POINT (23.56667 37.96667),42210.0,42010.0,PERAMA,GR,37.966667,23.566667,37.0,58.0,...,None,Y,Y,Y,Y,None,None,B,M,None


In [4]:
script2 = "select * from fishing_ports"

ports_fdf = gpd.GeoDataFrame.from_postgis(script2, con, geom_col="geom")

In [5]:
ports_fdf.head()

,id,geom,conceptid,preflabel,modified,altlabel,country,latitude,longitude,note
0,506,POINT (23.70000 37.95000),BSH155,Piraeus,2007-08-02 03:00:00,Piraeus,Greece,37.950000,23.700000,None
1,1384,POINT (23.73333 38.00000),BSH235,Athens,2007-08-02 03:00:00,Athens,Greece,38.000000,23.733330,None
2,2256,POINT (23.60000 37.96667),BSH3184,Limin Irakleous,2007-08-02 03:00:00,Limin Irakleous,Greece,37.966667,23.600000,None
3,2257,POINT (23.38333 37.96667),BSH3185,Pachi Oil Terminal,2007-08-02 03:00:00,Pachi Oil Terminal,Greece,37.966667,23.383333,None
4,2258,POINT (23.56667 37.96667),BSH3186,Perama,2007-08-02 03:00:00,Perama,Greece,37.966667,23.566667,None


In [6]:
frames = [ports_fdf, ports_wdf]

df_ports = pd.concat(frames, ignore_index=True)

df_ports.drop_duplicates(subset=['geom'], inplace=True, keep='first')

In [7]:
df_ports

,id,geom,conceptid,preflabel,modified,altlabel,country,latitude,longitude,note,...,elecrepair,provisions,water,fuel_oil,diesel,decksupply,eng_supply,repaircode,drydock,railway
0,506,POINT (23.70000 37.95000),BSH155,Piraeus,2007-08-02 03:00:00,Piraeus,Greece,37.950000,23.700000,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1384,POINT (23.73333 38.00000),BSH235,Athens,2007-08-02 03:00:00,Athens,Greece,38.000000,23.733330,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2256,POINT (23.60000 37.96667),BSH3184,Limin Irakleous,2007-08-02 03:00:00,Limin Irakleous,Greece,37.966667,23.600000,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2257,POINT (23.38333 37.96667),BSH3185,Pachi Oil Terminal,2007-08-02 03:00:00,Pachi Oil Terminal,Greece,37.966667,23.383333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2258,POINT (23.56667 37.96667),BSH3186,Perama,2007-08-02 03:00:00,Perama,Greece,37.966667,23.566667,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2259,POINT (23.48333 37.96667),BSH3187,Salamis Island,2007-10-16 16:20:12,Salamis Island,Greece,37.966667,23.483333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2262,POINT (23.33333 37.98333),BSH3190,Megara Oil Terminal,2007-08-02 03:00:00,Megara Oil Terminal,Greece,37.983333,23.333333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2263,POINT (23.53333 37.98333),BSH3191,Navstathmos,2007-08-02 03:00:00,Navstathmos,Greece,37.983333,23.533333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2265,POINT (23.41667 38.00000),BSH3193,Nea Peramos,2011-03-23 20:44:10,Nea Peramos,Greece,38.000000,23.416667,near Athens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2271,POINT (23.55000 38.03333),BSH3199,Elefsis,2007-10-16 16:20:12,Elefsis,Greece,38.033333,23.550000,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
points = st_visualizer.st_visualizer()
points.set_data(df.iloc[:3000, :].copy())
viz_express.plot_points_on_map(points, tools=['lasso_select'])

ports_viz = st_visualizer.st_visualizer()
ports_viz.set_data(df_ports.copy())
ports_viz.set_figure(points.figure)
ports_viz.create_source()

ports_viz.add_glyph(glyph_type='circle', size=10, color='crimson', alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label='Ports')

points.show_figures(notebook=True, notebook_url='http://localhost:8888')

Loading BokehJS ...

In [8]:
df_ports.geometry = df_ports.geometry.to_crs(epsg=2100).buffer(2000).to_crs(epsg=4326)

In [9]:
df_ports.geometry.head()

0    POLYGON ((23.72276 37.95006, 23.72266 37.94829...
1    POLYGON ((23.75611 38.00005, 23.75601 37.99828...
2    POLYGON ((23.62277 37.96674, 23.62267 37.96498...
3    POLYGON ((23.40610 37.96678, 23.40601 37.96502...
4    POLYGON ((23.58944 37.96675, 23.58934 37.96498...
Name: geom, dtype: geometry

In [10]:
df_ports

,id,geom,conceptid,preflabel,modified,altlabel,country,latitude,longitude,note,...,elecrepair,provisions,water,fuel_oil,diesel,decksupply,eng_supply,repaircode,drydock,railway
0,506,"POLYGON ((23.72276 37.95006, 23.72266 37.94829...",BSH155,Piraeus,2007-08-02 03:00:00,Piraeus,Greece,37.950000,23.700000,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1384,"POLYGON ((23.75611 38.00005, 23.75601 37.99828...",BSH235,Athens,2007-08-02 03:00:00,Athens,Greece,38.000000,23.733330,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2256,"POLYGON ((23.62277 37.96674, 23.62267 37.96498...",BSH3184,Limin Irakleous,2007-08-02 03:00:00,Limin Irakleous,Greece,37.966667,23.600000,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2257,"POLYGON ((23.40610 37.96678, 23.40601 37.96502...",BSH3185,Pachi Oil Terminal,2007-08-02 03:00:00,Pachi Oil Terminal,Greece,37.966667,23.383333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2258,"POLYGON ((23.58944 37.96675, 23.58934 37.96498...",BSH3186,Perama,2007-08-02 03:00:00,Perama,Greece,37.966667,23.566667,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2259,"POLYGON ((23.50610 37.96676, 23.50600 37.96500...",BSH3187,Salamis Island,2007-10-16 16:20:12,Salamis Island,Greece,37.966667,23.483333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2262,"POLYGON ((23.35611 37.98346, 23.35601 37.98169...",BSH3190,Megara Oil Terminal,2007-08-02 03:00:00,Megara Oil Terminal,Greece,37.983333,23.333333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2263,"POLYGON ((23.55611 37.98342, 23.55601 37.98165...",BSH3191,Navstathmos,2007-08-02 03:00:00,Navstathmos,Greece,37.983333,23.533333,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2265,"POLYGON ((23.43945 38.00011, 23.43935 37.99834...",BSH3193,Nea Peramos,2011-03-23 20:44:10,Nea Peramos,Greece,38.000000,23.416667,near Athens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2271,"POLYGON ((23.57279 38.03342, 23.57269 38.03165...",BSH3199,Elefsis,2007-10-16 16:20:12,Elefsis,Greece,38.033333,23.550000,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
points = st_visualizer.st_visualizer()
points.set_data(df.iloc[:1000, :].copy())
viz_express.plot_points_on_map(points, tools=['lasso_select'])

ports_viz = st_visualizer.st_visualizer()
ports_viz.set_data(df_ports.copy())
ports_viz.set_figure(points.figure)
ports_viz.create_source()

ports_viz.add_polygon(fill_color='crimson', line_color='crimson', legend_label='Ports')

points.show_figures(notebook=True, notebook_url='http://localhost:8888')

Loading BokehJS ...

In [89]:
%%time
sindex = df.sindex

Wall time: 13min 18s


In [90]:
%%time
points_within_geometry = pd.DataFrame()

for poly in df_ports.geometry:
    possible_matches_index = list(sindex.intersection(poly.bounds))
    possible_matches = df.iloc[possible_matches_index]
    
    precise_matches = possible_matches[possible_matches.intersects(poly)]
    points_within_geometry = points_within_geometry.append(precise_matches)

Wall time: 3min 39s


In [91]:
points_within_geometry = points_within_geometry.drop_duplicates(subset=['mmsi', 'timestamp'])

In [92]:
df.loc[:, 'traj_id'] = 0
df.loc[points_within_geometry.index, 'traj_id'] = -1

In [93]:
df.drop(['geom'], axis=1).to_csv('./data/unipi_ais_spat_seg.csv', header=True, index=False)

In [11]:
%%time
df = helper.getGeoDataFrame_v2(pd.read_csv('./data/unipi_ais_spat_seg.csv'), crs='epsg:4326')
df.sort_values('timestamp', inplace=True)

Wall time: 5min 28s


In [17]:
tmp = df.iloc[:5000, :].copy()
tmp.traj_id = tmp.traj_id.apply(str)

points = st_visualizer.st_visualizer()
points.set_data(tmp.copy())
points.create_canvas('Prototype Plot')

cmap = points.add_categorical_colormap(('crimson', 'royalblue'), 'traj_id')
points.add_glyph(color=cmap, legend_label='Ports')
points.add_map_tile(provider='CARTODBPOSITRON')

# points.add_hover_tooltips([('mmsi', '@mmsi'), 
#                            ('traj_id', '@traj_id'),
#                            ('timestamp', '@timestamp')])

ports_vis = st_visualizer.st_visualizer()
ports_vis.set_data(df_ports.copy())
ports_viz.set_figure(points.figure)
ports_viz.create_source()
ports_viz.add_polygon(fill_color=None, line_color='crimson', legend_label="Ports")

points.show_figures(notebook=True, notebook_url="http://localhost:8888")

Loading BokehJS ...

In [12]:
df = helper.create_trajectories(df.copy())

In [13]:
df.head()

,timestamp,type,mmsi,status,lon,lat,heading,turn,speed,course,timestamp_sec,velocity,bearing,acceleration,traj_id,geom,label
1344119,1520421611000,NaN,0,NaN,23.645050,37.860745,337.0,NaN,6.4,341.0,1.520422e+09,6.601579,333.290692,0.019794,0,POINT (23.64505 37.86075),0
1344128,1520421614000,NaN,0,NaN,23.645007,37.860830,337.0,NaN,6.5,337.9,1.520422e+09,6.542198,334.840587,0.044963,0,POINT (23.64501 37.86083),0
1344168,1520421625000,NaN,0,NaN,23.644858,37.861142,335.0,NaN,6.4,339.2,1.520422e+09,6.047604,331.806882,-0.015868,0,POINT (23.64486 37.86114),0
1344230,1520421646000,NaN,0,NaN,23.644565,37.861682,334.0,NaN,6.4,334.9,1.520422e+09,6.380839,330.772250,-0.522968,0,POINT (23.64457 37.86168),0
1344250,1520421652000,NaN,0,NaN,23.644473,37.861843,335.0,NaN,6.4,337.6,1.520422e+09,9.518650,330.074505,1.944526,0,POINT (23.64447 37.86184),0


In [14]:
df.shape

(6334721, 17)

In [21]:
tmp = df.iloc[:15000, :].copy()
tmp.traj_id = tmp.traj_id.apply(str)

points = st_visualizer.st_visualizer()
points.set_data(tmp.copy())
points.create_canvas('Prototype Plot')

cmap = points.add_categorical_colormap(('crimson', 'royalblue'), 'traj_id')
points.add_glyph(color=cmap, legend_label='Ports')
points.add_map_tile(provider='CARTODBPOSITRON')

# points.add_hover_tooltips([('mmsi', '@mmsi'), 
#                            ('traj_id', '@traj_id'),
#                            ('timestamp', '@timestamp')])

ports_vis = st_visualizer.st_visualizer()
ports_vis.set_data(df_ports.copy())
ports_viz.set_figure(points.figure)
ports_viz.create_source()
ports_viz.add_polygon(fill_color=None, line_color='crimson', legend_label="Ports")

points.show_figures(notebook=True, notebook_url="http://localhost:8888")

Loading BokehJS ...

In [22]:
df_single = df.loc[df.mmsi == 1193046].copy()
df_single.sort_values('timestamp', inplace=True)
df_single.reset_index(inplace=True, drop=True)

df_single = helper.fix_trajectories(df_single.copy())

(Initial) Number of segments: 3
(Final-Useful) Number of port-based segments produced: 2


In [23]:
tmp = df_single.copy()
tmp.traj_id = tmp.traj_id.apply(str)

points = st_visualizer.st_visualizer()
points.set_data(tmp.copy())
points.create_canvas('Prototype Plot')

cmap = points.add_categorical_colormap(('crimson', 'royalblue'), 'traj_id')
points.add_glyph(color=cmap, legend_label='Ports')
points.add_map_tile(provider='CARTODBPOSITRON')

# points.add_hover_tooltips([('mmsi', '@mmsi'), 
#                            ('traj_id', '@traj_id'),
#                            ('timestamp', '@timestamp')])

ports_vis = st_visualizer.st_visualizer()
ports_vis.set_data(df_ports.copy())
ports_viz.set_figure(points.figure)
ports_viz.create_source()
ports_viz.add_polygon(fill_color=None, line_color='crimson', legend_label="Ports")

points.show_figures(notebook=True, notebook_url="http://localhost:8888")

Loading BokehJS ...

In [24]:
df_single.sort_values('timestamp', inplace=True)
df_single2 = df_single.copy()
df_single2.loc[:, 'timestamp'] = df_single2.timestamp / 10**3

df_single2 = helper.temporal_segmentation(df_single2.copy())

(Initial) Number of port-based segments: 2
(Intermediate) Number of temporal-gap-based segments: 3
(Final-Useful) Number of trips produced: 2


In [25]:
tmp = df_single2.copy()
tmp.trip_id = tmp.trip_id.apply(str)
tmp.loc[:, 'date'] = pd.to_datetime(tmp.timestamp, unit='s').astype(str)

points = st_visualizer.st_visualizer()
points.set_data(tmp.copy())
points.create_canvas('Prototype Plot')

cmap = points.add_categorical_colormap(('crimson', 'royalblue'), 'trip_id')
points.add_glyph(color=cmap, legend_label='Ports')
points.add_map_tile(provider='CARTODBPOSITRON')

# points.add_hover_tooltips([('mmsi', '@mmsi'), 
#                            ('traj_id', '@traj_id'),
#                            ('timestamp', '@timestamp')])

ports_vis = st_visualizer.st_visualizer()
ports_vis.set_data(df_ports.copy())
ports_viz.set_figure(points.figure)
ports_viz.create_source()
ports_viz.add_polygon(fill_color=None, line_color='crimson', legend_label="Ports")

points.show_figures(notebook=True, notebook_url="http://localhost:8888")

Loading BokehJS ...

In [26]:
#temporal alignment
features = df_single2.drop(['index', 'geom'], axis=1).columns

df_single2.algn = df_single2.groupby(['mmsi', 'trip_id'], as_index=False).apply(lambda l: helper.temporal_alignment_v2(l, rate=1, method='linear', 
                                                                                                                        features=features, temporal_axis_name='datetime',
                                                                                                                        temporal_name='timestamp', temporal_unit='s')).reset_index(drop=True)

C:\anaconda3\envs\gis\lib\site-packages\geopandas\geodataframe.py:97: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


In [27]:
points = st_visualizer.st_visualizer()
points.set_data(df_single2.copy())
points.create_canvas('Plotorype Plot')
points.add_glyph(color='royalblue', legend_label='Non-Alligned Points')
points.add_map_tile(provider='CARTODBPOSITRON')

points_algn = st_visualizer.st_visualizer()
points_algn.set_data(df_single2.algn.copy())

points_algn.set_figure(points.figure)
points_algn.create_source()

points_algn.add_glyph(color='orangered', legend_label='Aligned Points')

points.figure.legend.click_policy = 'mute'
points.show_figures(notebook=True, notebook_url="http://localhost:8888")

Loading BokehJS ...

In [28]:
df_single2.head()

,index,timestamp,type,mmsi,status,lon,lat,heading,turn,speed,course,timestamp_sec,velocity,bearing,acceleration,traj_id,geom,label,trip_id
0,0,1.523365e+09,NaN,1193046,NaN,23.549597,37.916037,NaN,NaN,10.2,39.5,1.523365e+09,10.259682,43.671816,-0.003140,0,POINT (23.54960 37.91604),0,0
1,1,1.523365e+09,NaN,1193046,NaN,23.550332,37.916788,NaN,NaN,10.2,39.0,1.523365e+09,10.322486,43.286673,0.002561,0,POINT (23.55033 37.91679),0,0
2,2,1.523365e+09,NaN,1193046,NaN,23.551065,37.917548,NaN,NaN,10.2,38.5,1.523365e+09,10.271256,44.751128,-0.001590,0,POINT (23.55107 37.91755),0,0
3,3,1.523365e+09,NaN,1193046,NaN,23.551818,37.918290,NaN,NaN,10.2,39.4,1.523365e+09,10.303048,44.808528,0.001954,0,POINT (23.55182 37.91829),0,0
4,4,1.523365e+09,NaN,1193046,NaN,23.552575,37.919033,NaN,NaN,10.4,36.9,1.523365e+09,10.263976,45.149349,-0.001874,0,POINT (23.55258 37.91903),0,0


In [15]:
df.sort_values('timestamp', inplace=True)
df2 = df.copy()
df2.loc[:, 'timestamp'] = df2.timestamp / 10**3

df2 = helper.temporal_segmentation(df2.copy())

(Initial) Number of port-based segments: 2
(Intermediate) Number of temporal-gap-based segments: 4
(Final-Useful) Number of trips produced: 4


In [16]:
df2.shape

(6334721, 19)

In [17]:
df2.head()

,index,timestamp,type,mmsi,status,lon,lat,heading,turn,speed,course,timestamp_sec,velocity,bearing,acceleration,traj_id,geom,label,trip_id
0,0,1.519855e+09,NaN,636013190,NaN,23.539338,37.885675,44.0,NaN,1.4,15.5,1.519855e+09,1.384017,24.892345,-0.019672,0,POINT (23.53934 37.88567),0,2
1,3,1.519855e+09,NaN,237991700,NaN,23.546150,37.949950,NaN,NaN,0.0,268.0,1.519855e+09,0.000000,0.000000,0.000000,0,POINT (23.54615 37.94995),0,2
2,6,1.519855e+09,NaN,235100119,NaN,23.681892,37.932647,244.0,NaN,0.0,271.2,1.519855e+09,0.054364,280.586094,0.000281,0,POINT (23.68189 37.93265),0,2
3,8,1.519855e+09,NaN,256147000,NaN,23.535167,37.861398,296.0,NaN,0.3,214.5,1.519855e+09,0.394770,228.634116,0.001103,0,POINT (23.53517 37.86140),0,2
4,10,1.519855e+09,NaN,239945400,NaN,23.604900,37.920620,NaN,NaN,13.2,168.0,1.519855e+09,12.103464,166.692336,-0.107975,0,POINT (23.60490 37.92062),0,2


In [18]:
features = df2.drop(['index', 'geom'], axis=1).columns

df2.algn = df2.groupby(['mmsi', 'trip_id'], as_index=False).apply(lambda l: helper.temporal_alignment_v2(l, rate=1, method='linear', 
                                                                                                                        features=features, temporal_axis_name='datetime',
                                                                                                                        temporal_name='timestamp', temporal_unit='s')).reset_index(drop=True)

C:\anaconda3\envs\gis\lib\site-packages\geopandas\geodataframe.py:97: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


In [19]:
df2.head()

,index,timestamp,type,mmsi,status,lon,lat,heading,turn,speed,course,timestamp_sec,velocity,bearing,acceleration,traj_id,geom,label,trip_id
0,0,1.519855e+09,NaN,636013190,NaN,23.539338,37.885675,44.0,NaN,1.4,15.5,1.519855e+09,1.384017,24.892345,-0.019672,0,POINT (23.53934 37.88567),0,2
1,3,1.519855e+09,NaN,237991700,NaN,23.546150,37.949950,NaN,NaN,0.0,268.0,1.519855e+09,0.000000,0.000000,0.000000,0,POINT (23.54615 37.94995),0,2
2,6,1.519855e+09,NaN,235100119,NaN,23.681892,37.932647,244.0,NaN,0.0,271.2,1.519855e+09,0.054364,280.586094,0.000281,0,POINT (23.68189 37.93265),0,2
3,8,1.519855e+09,NaN,256147000,NaN,23.535167,37.861398,296.0,NaN,0.3,214.5,1.519855e+09,0.394770,228.634116,0.001103,0,POINT (23.53517 37.86140),0,2
4,10,1.519855e+09,NaN,239945400,NaN,23.604900,37.920620,NaN,NaN,13.2,168.0,1.519855e+09,12.103464,166.692336,-0.107975,0,POINT (23.60490 37.92062),0,2


In [20]:
df2.drop(['geom', 'index'], axis=1).to_csv('./data/traj_final.csv', header=True, index=False)